In [15]:
contactSheets = [x.replace(os.sep,'/') for x in glob(path)]
cs_splits = [x.split('/') for x in contactSheets]
cs_splitspds = pd.Series([x.split('/') for x in contactSheets])

df = pd.DataFrame(cs_splits)
# df['name'] = df[4]
# df

df = pd.DataFrame([x.replace(os.sep,'/').split('/') for x in glob(path)])
df
# unique in column
print(df[4].unique())

# unique+count in column
print(df[4].value_counts())



['CamilaMazurek' 'ChrisHemsworth' 'JasonMomoa' 'KamilaTen' 'LucyMae'
 'ManuelTucker' 'MonstaXJooheon' 'PaulSamatar' 'PhilipAn' 'RonaldNelson'
 'AmandaMoore' 'AliceRivera' 'WandaEdwards' 'VeronicaYoung' 'LouisPrice'
 'SimonYuen']
SimonYuen         26
CamilaMazurek     18
RonaldNelson      16
ChrisHemsworth    14
KamilaTen         14
LucyMae           14
ManuelTucker      14
MonstaXJooheon    14
PaulSamatar       14
PhilipAn          14
VeronicaYoung     14
LouisPrice        14
JasonMomoa        13
AmandaMoore       12
WandaEdwards      11
AliceRivera        2
Name: 4, dtype: int64


In [1]:
from glob import glob
import pandas as pd

path = "G:/Shared drives/TriplegangersGroom_ext/Groom_INTERNAL/*/maya/base_delta/images/*/contactSheet.png"

df = pd.DataFrame([x.replace(os.sep,'/').split('/') for x in glob(path)])
df
# unique in column
print(df[4].unique())

# unique+count in column
print(df[4].value_counts())

['CamilaMazurek' 'ChrisHemsworth' 'JasonMomoa' 'KamilaTen' 'LucyMae'
 'ManuelTucker' 'MonstaXJooheon' 'PaulSamatar' 'PhilipAn' 'RonaldNelson'
 'AmandaMoore' 'AliceRivera' 'WandaEdwards' 'VeronicaYoung' 'LouisPrice'
 'SimonYuen']
SimonYuen         26
CamilaMazurek     18
RonaldNelson      16
ChrisHemsworth    14
KamilaTen         14
LucyMae           14
ManuelTucker      14
MonstaXJooheon    14
PaulSamatar       14
PhilipAn          14
VeronicaYoung     14
LouisPrice        14
JasonMomoa        13
AmandaMoore       12
WandaEdwards      11
AliceRivera        2
Name: 4, dtype: int64
